Analyzing G25 corrdinates of ancient DNA samples using graphs

In [2]:
require(rio)
require(tidyverse)
require(cowplot)
require(dbscan)
require(igraph)
require(tidygraph)
require(ggraph)
require(pals)

Load the data for modern and ancient G25 population averages

In [3]:
avgs = as_tibble(import("../Genetics/G25/Data/TXT/Global25_PCA_pop_averages_scaled.txt"))
#avgs %>% sample_n(10)
modavgs = as_tibble(import("../Genetics/G25/Data/TXT/Global25_PCA_modern_pop_averages_scaled.txt"))
#modavgs %>% sample_n(10)

Optionally, filter the dataset down

In [3]:
str_extract(avgs$V1,"^\\w+?(?=_)") %>% unique(sort = TRUE)

[1] "ARG"         "ARM"         "AUS"         "AUT"         "AZE"        
  [6] "Baltic"      "BEL"         "Bell"        "BGR"         "BHS"        
 [11] "BLZ"         "BRA"         "BWA"         "CAN"         "Canary"     
 [16] "Channel"     "CHE"         "CHL"         "CHN"         "CMR"        
 [21] "COG"         "Corded"      "CUW"         "CZE"         "DEU"        
 [26] "DNK"         "DOM"         "EGY"         "England"     "ETH"        
 [31] "FIN"         "FRA"         "GEO"         "Gepidian"    "GRC"        
 [36] "Greater"     "GUM"         "HRV"         "HTI"         "HUN"        
 [41] "Hun"         "Iberia"      "IDN"         "IND"         "IRL"        
 [46] "IRN"         "ISL"         "Isle"        "ITA"         "JPN"        
 [51] "KAZ"         "KEN"         "KGZ"         "KOR"         NA           
 [56] "LAO"         "Levant"      "LUX"         "MAR"         "MDA"        
 [61] "MEX"         "MKD"         "MNG"         "MWI"         "MYS"        
 [66] "NLD"         "NOR"         "NPL"         "Orkney"      "Ostrogothic"
 [71] "PAK"         "PAN"         "PER"         "POL"         "PYF"        
 [76] "ROU"         "RUS"         "Saka"        "Sargat"      "Sarmatian"  
 [81] "Scotland"    "Scythian"    "SDN"         "SRB"         "SVK"        
 [86] "SVN"         "SWE"         "SYR"         "THA"         "TJK"        
 [91] "TKM"         "TON"         "TUR"         "TWN"         "TZA"        
 [96] "UGA"         "UKR"         "USA"         "UZB"         "VEN"        
[101] "VK2020"      "VNM"         "VUT"         "Wales"       "Yamnaya"    
[106] "ZAF"

In [4]:
str_extract(modavgs$V1,"^\\w+?$") %>% unique(sort = TRUE)

[1] "Abazin"                               
  [2] "Abkhasian"                            
  [3] "Abkhasian_Gudauta"                    
  [4] "Adygei"                               
  [5] "Aeta"                                 
  [6] "Afrikaner"                            
  [7] "Agta"                                 
  [8] "Ahiska"                               
  [9] "Akha"                                 
 [10] "Akhvakh"                              
 [11] "Alawite"                              
 [12] "Albanian"                             
 [13] "Alevi_Dersim"                         
 [14] "Algerian"                             
 [15] "Altaian"                              
 [16] "Amerindian_North"                     
 [17] "Ami"                                  
 [18] "Andian_A"                             
 [19] "Andian_B"                             
 [20] "Arain"                                
 [21] "Armenian_Aintab"                      
 [22] "Armenian_Ararat"                      
 [23] "Armenian_Artsakh"                     
 [24] "Armenian_Erzurum"                     
 [25] "Armenian_Gesaria"                     
 [26] "Armenian_Gurin"                       
 [27] "Armenian_Hemsheni"                    
 [28] "Armenian_Parspatunik"                 
 [29] "Armenian_Syunik"                      
 [30] "Armenian_Urfa"                        
 [31] "Arora"                                
 [32] "Ashkenazi_Belarussia"                 
 [33] "Ashkenazi_Germany"                    
 [34] "Ashkenazi_Lithuania"                  
 [35] "Ashkenazi_Poland"                     
 [36] "Ashkenazi_Russia"                     
 [37] "Ashkenazi_Ukraine"                    
 [38] "Assyrian"                             
 [39] "Asur"                                 
 [40] "Atayal"                               
 [41] "Australian"                           
 [42] "Austrian"                             
 [43] "Avar"                                 
 [44] "Awan"                                 
 [45] "Aymara"                               
 [46] "Azerbaijani_Dagestan"                 
 [47] "Azerbaijani_Iran"                     
 [48] "Azerbaijani_Republic"                 
 [49] "Azerbaijani_Republic_Gabala"          
 [50] "Azerbaijani_Republic_Shaki"           
 [51] "Azerbaijani_Turkey"                   
 [52] "Bagvalin"                             
 [53] "Bagvalin_o"                           
 [54] "Bahun"                                
 [55] "Bahun_o"                              
 [56] "Bai"                                  
 [57] "Baiku_Yao_Guizhou"                    
 [58] "Bajo"                                 
 [59] "Baka"                                 
 [60] "Bakola"                               
 [61] "Balija"                               
 [62] "Balkar"                               
 [63] "Balochi"                              
 [64] "Balti"                                
 [65] "Balti_o"                              
 [66] "Baniya_Gujarat"                       
 [67] "Baniya_Punjab"                        
 [68] "Baniya_Uttar_Pradesh_Gupta"           
 [69] "Bantu_Kenya"                          
 [70] NA                                     
 [71] "Baoan"                                
 [72] "Bashkir"                              
 [73] "Basque_Araba"                         
 [74] "Basque_Baztan"                        
 [75] "Basque_Biscay"                        
 [76] "Basque_French"                        
 [77] "Basque_Gipuzkoa"                      
 [78] "Basque_Gipuzkoa_Southwest"            
 [79] "Basque_Lower_Navarre"                 
 [80] "Basque_Navarre_Center"                
 [81] "Basque_Navarre_North"                 
 [82] "Basque_Roncal"                        
 [83] "Basque_Soule"                         
 [84] "Basque_Spanish"                       
 [85] "Batak"                                
 [86] "BedouinA"                             
 [87] "BedouinB"                             


In [25]:
str_extract(modavgs$V1,"^\\w+?$") %>% unique(sort = TRUE) %>% as.character() %>% paste0(collapse = "|")

[1] "Abazin|Abkhasian|Abkhasian_Gudauta|Adygei|Aeta|Afrikaner|Agta|Ahiska|Akha|Akhvakh|Alawite|Albanian|Alevi_Dersim|Algerian|Altaian|Amerindian_North|Ami|Andian_A|Andian_B|Arain|Armenian_Aintab|Armenian_Ararat|Armenian_Artsakh|Armenian_Erzurum|Armenian_Gesaria|Armenian_Gurin|Armenian_Hemsheni|Armenian_Parspatunik|Armenian_Syunik|Armenian_Urfa|Arora|Ashkenazi_Belarussia|Ashkenazi_Germany|Ashkenazi_Lithuania|Ashkenazi_Poland|Ashkenazi_Russia|Ashkenazi_Ukraine|Assyrian|Asur|Atayal|Australian|Austrian|Avar|Awan|Aymara|Azerbaijani_Dagestan|Azerbaijani_Iran|Azerbaijani_Republic|Azerbaijani_Republic_Gabala|Azerbaijani_Republic_Shaki|Azerbaijani_Turkey|Bagvalin|Bagvalin_o|Bahun|Bahun_o|Bai|Baiku_Yao_Guizhou|Bajo|Baka|Bakola|Balija|Balkar|Balochi|Balti|Balti_o|Baniya_Gujarat|Baniya_Punjab|Baniya_Uttar_Pradesh_Gupta|Bantu_Kenya|NA|Baoan|Bashkir|Basque_Araba|Basque_Baztan|Basque_Biscay|Basque_French|Basque_Gipuzkoa|Basque_Gipuzkoa_Southwest|Basque_Lower_Navarre|Basque_Navarre_Center|Basque_Navarre_North|Basque_Roncal|Basque_Soule|Basque_Spanish|Batak|BedouinA|BedouinB|Bedzan|Belarusian|BelgianA|BelgianB|BelgianC|Bengali_Bangladesh|Bengali_Bangladesh_SouthEast|Bengali_Bangladesh_Sylhet|Berber_Algeria|Berber_MAR_ERR|Berber_MAR_TIZ|Berber_Tunisia_Chen|Berber_Tunisia_Sen|Besermyan|Bhumihar_Bihar|Bhumij|Biaka|Birhor|Bitonga|Blang|Bolivian_Cochabamba|Bolivian_LaPaz|Bolivian_Pando|Bonan|Bonda|Bosnian|Brahmin_Gujarat|Brahmin_Gujarat_Audichya|Brahmin_Gujarat_Bardai|Brahmin_Gujarat_Nagar|Brahmin_Gujarat_o|Brahmin_Himachal_Pradesh_West|Brahmin_Jammu_Dogra|Brahmin_Karnataka_Tulu|Brahmin_Kerala_Nambudiri|Brahmin_Konkani_Catholic|Brahmin_Kumaon|Brahmin_Manipuri|Brahmin_Rajasthan|Brahmin_Tamil_Nadu|Brahmin_Tamil_Nadu_Iyengar|Brahmin_Tamil_Nadu_Iyer|Brahmin_Telugu|Brahmin_Telugu_Niyogi|Brahmin_Telugu_Vaidiki|Brahmin_UP_Awadh_Saryupareen|Brahmin_UP_Kanyakubja|Brahmin_UP_Lucknow|Brahmin_UP_Lucknow_o|Brahmin_Uttar_Pradesh_East|Brahmin_Uttar_Pradesh_East_o|Brahmin_West_Bengal|Brahui|Bukharian_Jew|Bulala|Bulgarian|Bunt_Kerala|Burmese|Burusho|Buryat|Cachi|Cambodian|Cameroon_Aghem|Cameroon_Bafut|Cameroon_Bakoko|Cameroon_Bangwa|Cameroon_Mbo|Chamalin|Chamar_Uttar_Pradesh|Chamar_Uttar_Pradesh_o|Changana|Changshan_Yao_Guizhou|Chechen|Chenchu|Cherkes|Chipewyan|Chopi|Chukchi|Chuvash|Circassian|Cochin_Jew_A|Cochin_Jew_B|Colla|Cossack_Kuban|Cossack_Ukrainian|Cree|Croatian|Cypriot|Czech|Dai|Damai|Danish|Darginian|Datog|Daur|Dharkar|Dinka|Dolgan|Dong_Guizhou|Dong_Hunan|Dongxiang|Druze|Dungan|Dusadh|Dusun|Dutch|Egyptian|Elmolo|EmiratiA|EmiratiB|EmiratiC|English|English_Cornwall|Eritrean|Erzya|Esan_Nigeria|Eskimo|Eskimo_Chaplin|Eskimo_Naukan|Eskimo_Sireniki|Estonian|Ethiopian_Afar|Ethiopian_Agaw|Ethiopian_Amhara|Ethiopian_Anuak|Ethiopian_Ari|Ethiopian_Ari_blacksmith|Ethiopian_Ari_cultivator|Ethiopian_Gumuz|Ethiopian_Jew|Ethiopian_Mursi|Ethiopian_Oromo|Ethiopian_Tigray|Ethiopian_Wolayta|Even|Evenk|Ezhava|Ezid|Finnish_Central|Finnish_East|Finnish_North|Finnish_Southeast|Finnish_Southwest|French_Alsace|French_Auvergne|French_Bearn|French_Bigorre|French_Brittany|French_Chalosse|French_Corsica|French_Nord|French_Occitanie|French_Paris|French_Provence|French_South|Fulani|Fulani_Ziniare|Gadaba|Gagauz|Gambian|Ganguela|Gelao|Georgian_Ajar|Georgian_Guria|Georgian_Imer|Georgian_Javakheti|Georgian_Jew|Georgian_Kakh|Georgian_Kart|Georgian_Khevs|Georgian_Laz|Georgian_Lechkhumi|Georgian_Megr|Georgian_Meskheti|Georgian_Mtiuleti|Georgian_NorthEast|Georgian_Ratcha|Georgian_Samtckhe|Georgian_Svaneti|Georgian_Tush|Georgian_West|German|German_East|German_Erlangen|German_Hamburg|Gond|Greek_Achaea|Greek_Apulia|Greek_Arcadia|Greek_Argolis|Greek_Cappadocia|Greek_Central_Anatolia|Greek_Central_Macedonia|Greek_Corinthia|Greek_Crete|Greek_Crete_Chania|Greek_Crete_Heraklion|Greek_Crete_Lasithi|Greek_Cyclades_Amorgos|Greek_Cyclades_Kea|Greek_Cyclades_Milos|Greek_Cyclades_Tinos|Greek_Deep_Mani|Greek_Dodecanese|Greek_Dodecanese_Rhodes|Greek_East_Macedonia_and_Thrace|Greek_East_Taygetos|Greek_Elis|Greek

In [4]:
avgs = avgs %>% dplyr::filter(!str_detect(V1,"^(ARG|AUS|BHS|BLZ|BRA|BWA|CAN|CHL|CHN|CMR|COG|CUW|DOM|ETH|GUM|HTI|IDN|IND|JPN|KAZ|KEN|KGZ|KOR|LAO|MEX|MNG|MWI|MYS)"))
avgs = avgs %>% dplyr::filter(!str_detect(V1,"^(NPL|PAK|PAN|PER|PYF|Saka|Sarg|SDN|THA|TJK|TKM|TON|TWN|TZA|UGA|USA|UZB|VEN|VNM|VUT|ZAF)"))
avgs = avgs %>% dplyr::filter(!str_detect(V1,"_o$"))
avgs = avgs %>% dplyr::filter(!str_detect(V1,"low"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Abazin|Abkhasian|Adygei|Aeta|Afrikaner|Agta|Ahiska|Akha|Akhvakh|Alawite|Alevi|Algerian|Altaian|Amerindian|Ami|Andian|Arain)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Armenian_[AGHP]|Arora|Assyrian|Asur|Atayal|Australian|Awan|Aymara|Azerbaijani_R|Basque_[ABGLR]|Batak|Bedzan|Bengali|Berber_[AM])"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Arora|Assyrian|Asur|Atayal|Australian|Awan|Aymara|Azerbaijani_R|Batak|Bedzan|Bengali|Bagvalin|Bahun|Baiku|Balti|Baniya|Bantu)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Besermyan|Bhumi|Biaka|Birhor|Bitonga|Blang|Bolivian|Bonan|Bonda|Brahmin|Brahui|Bulala|Bunt|Burmese|Burusho|Buryat|Cachi|Cambodian)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Cameroon|Chama|Chan|Chen|Chipewyan|Chopi|Chukchi|Chuvash|Circassian|Cochin|Colla|Cree|Dai|Damai|Darginian|Datog|Daur|Dharkar|Dinka)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Dolgan|Dong|Dungan|Dusadh|Dusun|Elmolo|Eritrean|Esan_Nigeria|Eskimo|Ethiopian|Even|Evenk|Ezhava|Ezid)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(French_Au|French_B[ei]|French_[COP]|Fulani|Gadaba|Gagauz|Gambian|Ganguela|Gelao|Georgian_[GIJKLMRST]|Gond|Greek_[ADEIKLMNSTW])"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Gujar|Gurung|Hadza|Hakkipikki|Han|Hawaiian|Hazara|Hezhen|Hinukh|Hmong|Ho|Htin_Mal|Hui|Hunzib|Igbo|Igorot|Indonesian|Iranian_[BJLMP])"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Iraq|Irula|Italian_[ABEFJLM]|Itelmen|Jamatia|Japanese|Jarawa|Jat|Jehai|Ju_hoan_North|Juang|Kaba|Kadar|Kaitag|Kalash|Kalmyk|Kamboj)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Irula|Itelmen|Jamatia|Japanese|Jarawa|Jat|Jehai|Ju_hoan_North|Juang|Kaba|Kadar|Kaitag|Kalash|Kalmyk|Kamboj)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Kamma|Kanjar|Karachay|Karaite_Egypt|Karakalpak|Karata|Karen_Sgaw|Karitiana|Kashmiri|Kazakh|Ket|Khakass|Khamnegan|Khanty|Khatri)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Khmer|Kho|Kikuyu|Kinh_Vietnam|Kirghiz|Knanaya|Kohistani|Koinanbe|Kol|Komi|Kongo|Konkani|Korean|Korwa|Koryak|Kosipe)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Kshatriya|Kubachinian|Kumyk|Kurdish|Kurichiya|Kurumba|Kusunda|Kuy_Suay|Lahu|Laka|Lao|Lawa|Lebbo|Lemande|Lezgin|Li|Libyan)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Lithuanian_[RP]|Luhya_Kenya|Luo|Luzon|Macedonian|Mada|Madagascar|Madiga|Magar|Makhuwa|Makrani|Mala|Malay|Manchu|Mandenka|Maniq)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Maniyani_Kerala|Mansi|Manyika|Maonan|Maori|Maratha|Mari|Masai|Mayan|Mbuti|Mende_Sierra_Leone|Miao|Mix|Mlabri|Mogush|Mon|Moroccan_)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Mountain_|Mozabite|Mulam|Murut|Mwani|Nahua|Nair|Nanai|Nasoi|Nasrani|Naxi|Ndau|Negidal|Nenets|Nepali|Newar|Nganassan|Ngumba|Nivkh|Nogai)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(North|Nyah_Kur|Nyan|Ogiek|Onge|Orcadian|Oroqen|Palestinian_|Pallan|Pamiri|Paniya|Papuan|Parsi|Pashtun|Pathan_Bhopal|Piapoco|Pima)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Piramalai_Kallar|Poduval_Kerala|Polish_|Pul|Pumi|Punj|Qiang|Qing|Quechua|Rai|Raj|Ratlub|Reddy|Relli|Rendille|Riang|Rohingya)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Roma_|Romaniote|Ronga|Ror|Rumelia_East|Russian_[BKRSTVY]|Saharawi|Sakha|Sakilli|Salar|Saliya_Kerala|Samaritan|Sandawe|Santhal)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Satnami_Chhattisgarh|Saudi[AB]|Selkup|Sena|Sengwer|She|Shor|Sindhi|Somali|Spanish_[ABCEGLPS]|Sri_Lankan|Sudanese|Surui)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Syed|Tabasaran|Tai_Lue|Tajik|Talysh_Azerbaijan|Tamang|Tamil_Sri_Lanka|Tarkhan_Muslim|Tat|Telugu|Thai|Tharu|Thiyya|Tibetan)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Tikar_South|Tindal|Tlingit|Todzin|Tripuri|Tsez|Tswa|Turkish_[ABDEGKNS]|Turkmen|Tuvinian|Tyagi|Udi|Udmurt|Ulchi|Umbundu)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Uttar|Uygur|Uzbek|Vaniya_Kerala|Velama|Vellalar|Vishwakarma_Kerala|Vizayan|Wa|Welsh|Wichi|Xibo|Yadav_Telugu|Yakut|Yao|Yemenite)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"^(Yi|Yoruba|Yugur|Yukagir|Yukpa|Yuku|Zapotec|Zhuang)"))
modavgs = modavgs %>% dplyr::filter(!str_detect(V1,"_o$"))

In [5]:
avgs$V1 %>% unique(sort = TRUE)
modavgs$V1 %>% unique(sort = TRUE)

[1] "ARM_Areni_C"                        
  [2] "ARM_LBA"                            
  [3] "ARM_Lchashen_MBA"                   
  [4] "ARM_MBA"                            
  [5] "AUT_IA_La_Tene"                     
  [6] "AUT_LBK_N"                          
  [7] "Baltic_EST_BA"                      
  [8] "Baltic_EST_IA"                      
  [9] "Baltic_EST_MA"                      
 [10] "Baltic_EST_Narva"                   
 [11] "Baltic_LTU_BA"                      
 [12] "Baltic_LTU_Meso"                    
 [13] "Baltic_LTU_Narva"                   
 [14] "Baltic_LVA_BA"                      
 [15] "Baltic_LVA_HG"                      
 [16] "Baltic_LVA_MN"                      
 [17] "BEL_GoyetQ116-1"                    
 [18] "BEL_GoyetQ2"                        
 [19] "Bell_Beaker_Bavaria"                
 [20] "Bell_Beaker_CHE"                    
 [21] "Bell_Beaker_CZE"                    
 [22] "Bell_Beaker_CZE_early"              
 [23] "Bell_Beaker_CZE_late"               
 [24] "Bell_Beaker_England"                
 [25] "Bell_Beaker_England_EBA"            
 [26] "Bell_Beaker_England_highEEF"        
 [27] "Bell_Beaker_England_highWHG"        
 [28] "Bell_Beaker_FRA"                    
 [29] "Bell_Beaker_FRA_C"                  
 [30] "Bell_Beaker_HUN"                    
 [31] "Bell_Beaker_HUN_EBA"                
 [32] "Bell_Beaker_Iberia"                 
 [33] "Bell_Beaker_Iberia_C"               
 [34] "Bell_Beaker_ITA"                    
 [35] "Bell_Beaker_Mittelelbe-Saale"       
 [36] "Bell_Beaker_Mittelelbe-Saale_contam"
 [37] "Bell_Beaker_NLD"                    
 [38] "Bell_Beaker_POL"                    
 [39] "Bell_Beaker_Rhine-Main"             
 [40] "Bell_Beaker_Scotland"               
 [41] "BGR_Bacho_Kiro_MUP"                 
 [42] "BGR_Beli_Breyag_EBA"                
 [43] "BGR_C"                              
 [44] "BGR_Dzhulyunitsa_N"                 
 [45] "BGR_EBA"                            
 [46] "BGR_EBA_contam"                     
 [47] "BGR_IA"                             
 [48] "BGR_Krepost_N"                      
 [49] "BGR_Late_C"                         
 [50] "BGR_Middle_C"                       
 [51] "BGR_MLBA"                           
 [52] "BGR_MP_N"                           
 [53] "BGR_N"                              
 [54] "BGR_Varna_C"                        
 [55] "Canary_Islands_Guanche"             
 [56] "Channel_Islands_EIA"                
 [57] "Channel_Islands_IA"                 
 [58] "Channel_Islands_LN"                 
 [59] "Channel_Islands_MN"                 
 [60] "CHE_EBA"                            
 [61] "CHE_FN"                             
 [62] "CHE_FN_steppe_contam"               
 [63] "CHE_IA"                             
 [64] "CHE_LN"                             
 [65] "CHE_LN_contam"                      
 [66] "CHE_LN_steppe"                      
 [67] "CHE_MN"                             
 [68] "Corded_Ware_Baltic"                 
 [69] "Corded_Ware_Baltic_early"           
 [70] "Corded_Ware_CHE"                    
 [71] "Corded_Ware_CZE"                    
 [72] "Corded_Ware_CZE_early"              
 [73] "Corded_Ware_CZE_late"               
 [74] "Corded_Ware_CZE_noSteppe"           
 [75] "Corded_Ware_DEU"                    
 [76] "Corded_Ware_POL"                    
 [77] "Corded_Ware_POL_early"              
 [78] "Corded_Ware_Proto-Unetice_POL"      
 [79] "CZE_Bilina_BA"                      
 [80] "CZE_C"                              
 [81] "CZE_C_oSteppe"                      
 [82] "CZE_Early_Slav"                     
 [83] "CZE_EBA_Unetice"                    
 [84] "CZE_EE"                             
 [85] "CZE_EN_LBK"                         
 [86] "CZE_Hallstatt_Bylany"               
 [87] "CZE_IA_Hallstatt"                   
 [88] "CZE_IA_La_Tene"                     
 [89] "CZE_IA_La_Tene_Hallstatt"           
 [90] "CZE_IA_La_Tene_oFennoscandian"      
 [91] "CZE_Krems_UP"                      

[1] "Albanian"                    "Armenian_Erzurum"           
  [3] "Armenian_Syunik"             "Armenian_Urfa"              
  [5] "Ashkenazi_Belarussia"        "Ashkenazi_Germany"          
  [7] "Ashkenazi_Lithuania"         "Ashkenazi_Poland"           
  [9] "Ashkenazi_Russia"            "Ashkenazi_Ukraine"          
 [11] "Austrian"                    "Avar"                       
 [13] "Azerbaijani_Dagestan"        "Azerbaijani_Iran"           
 [15] "Azerbaijani_Turkey"          "Bai"                        
 [17] "Bajo"                        "Baka"                       
 [19] "Bakola"                      "Balija"                     
 [21] "Balkar"                      "Balochi"                    
 [23] "Baoan"                       "Bashkir"                    
 [25] "Basque_French"               "Basque_Navarre_Center"      
 [27] "Basque_Navarre_North"        "Basque_Soule"               
 [29] "Basque_Spanish"              "BedouinA"                   
 [31] "BedouinB"                    "Belarusian"                 
 [33] "BelgianA"                    "BelgianB"                   
 [35] "BelgianC"                    "Berber_Tunisia_Chen"        
 [37] "Berber_Tunisia_Sen"          "Bosnian"                    
 [39] "Bukharian_Jew"               "Bulgarian"                  
 [41] "Chechen"                     "Cherkes"                    
 [43] "Cossack_Kuban"               "Cossack_Ukrainian"          
 [45] "Croatian"                    "Cypriot"                    
 [47] "Czech"                       "Danish"                     
 [49] "Druze"                       "Dutch"                      
 [51] "Egyptian"                    "EmiratiA"                   
 [53] "EmiratiB"                    "EmiratiC"                   
 [55] "English"                     "English_Cornwall"           
 [57] "Erzya"                       "Estonian"                   
 [59] "Finnish_Central"             "Finnish_East"               
 [61] "Finnish_North"               "Finnish_Southeast"          
 [63] "Finnish_Southwest"           "French_Alsace"              
 [65] "French_Brittany"             "French_Nord"                
 [67] "French_Seine-Maritime"       "French_South"               
 [69] "Georgian_Ajar"               "Georgian_NorthEast"         
 [71] "Georgian_West"               "German"                     
 [73] "German_East"                 "German_Erlangen"            
 [75] "German_Hamburg"              "Greek_Cappadocia"           
 [77] "Greek_Central_Anatolia"      "Greek_Central_Macedonia"    
 [79] "Greek_Corinthia"             "Greek_Crete"                
 [81] "Greek_Crete_Chania"          "Greek_Crete_Heraklion"      
 [83] "Greek_Crete_Lasithi"         "Greek_Cyclades_Amorgos"     
 [85] "Greek_Cyclades_Kea"          "Greek_Cyclades_Milos"       
 [87] "Greek_Cyclades_Tinos"        "Greek_Peloponnese"          
 [89] "Greenlander_East"            "Greenlander_West"           
 [91] "Hungarian"                   "Icelandic"                  
 [93] "Ingrian"                     "Ingushian"                  
 [95] "Iranian"                     "Iranian_Fars"               
 [97] "Iranian_Zoroastrian"         "Irish"                      
 [99] "Italian_Calabria"            "Italian_Campania"           
[101] "Italian_Northeast"           "Italian_Piedmont"           
[103] "Italian_Trentino_Alto_Adige" "Italian_Tuscany"            
[105] "Italian_Umbria"              "Italian_Veneto"             
[107] "Jordanian"                   "Karelian"                   
[109] "Lak"                         "Latvian"                    
[111] "Lebanese_Christian"          "Lebanese_Druze"             
[113] "Lebanese_Muslim"             "Maltese"                    
[115] "Moksha"                      "Moldovan"                   
[117] "Moroccan"                    "Norwegian"                  
[119] "Ossetian"                    "Palestinian"                
[121] "Polish"                      "Portu

In [6]:
myavgs = avgs %>% select(-V1)
row.names(myavgs) = avgs$V1
mymodavgs = modavgs %>% select(-V1)
row.names(mymodavgs) = modavgs$V1

Warning message:
“Setting row names on a tibble is deprecated.”
Warning message:
“Setting row names on a tibble is deprecated.”


Combine all data

In [7]:
allavgs = bind_rows(myavgs,mymodavgs)

Find nearest neighbors for each row

In [50]:
mynn = kNN(allavgs,k = nrow(allavgs)-1)
mynn

k-nearest neighbors for 995 objects (k=994).
Available fields: dist, id, k, sort

In [51]:
mynn_d = rownames_to_column(as.data.frame(round(mynn$dist,2))) %>% as_tibble() %>% mutate(`0` = 0,.after = rowname)
mynn = rownames_to_column(as.data.frame(mynn$id)) %>% as_tibble() %>% mutate(`0` = row_number(),.after = rowname)

In [52]:
mynn %>% sample_n(5)
mynn_d %>% sample_n(5)

rowname,0,1,2,3,4,5,6,7,8,⋯,985,986,987,988,989,990,991,992,993,994
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
UKR_Trypillia_En,753,52,104,30,99,714,84,103,540,⋯,559,653,656,554,598,586,236,237,841,840
Baltic_EST_Narva,10,710,711,547,15,750,546,13,545,⋯,559,598,586,656,978,554,236,237,841,840
VK2020_DNK_Funen_VA,754,802,801,761,763,870,757,756,758,⋯,978,559,656,598,554,586,236,237,841,840
German_Hamburg,897,757,802,894,896,895,801,754,870,⋯,638,559,656,598,554,586,236,237,841,840
SVN_EIA,698,287,5,296,970,687,925,94,963,⋯,653,559,656,598,554,586,236,237,841,840


rowname,0,1,2,3,4,5,6,7,8,⋯,985,986,987,988,989,990,991,992,993,994
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Italian_Northeast,0,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,⋯,0.62,0.62,0.62,0.62,0.62,0.63,0.68,0.69,0.80,0.81
TUR_Catalhoyuk_N_Ceramic,0,0.04,0.05,0.05,0.05,0.05,0.06,0.06,0.06,⋯,0.66,0.66,0.66,0.66,0.66,0.68,0.73,0.74,0.80,0.80
CZE_EE,0,0.01,0.01,0.02,0.02,0.02,0.02,0.02,0.02,⋯,0.66,0.66,0.66,0.66,0.66,0.67,0.73,0.74,0.82,0.82
Balochi,0,0.07,0.09,0.09,0.10,0.10,0.10,0.10,0.11,⋯,0.56,0.56,0.56,0.56,0.56,0.58,0.63,0.65,0.76,0.76
Levant_Beirut_IAII,0,0.02,0.02,0.02,0.02,0.03,0.03,0.03,0.03,⋯,0.63,0.64,0.64,0.64,0.64,0.65,0.71,0.72,0.79,0.79


Pivot and clean up the data

In [53]:
mynames = mynn %>% select(rowname) %>% mutate(val = row_number())

In [54]:
mynn = mynn %>% pivot_longer(cols = matches("^\\d"),names_to = "rank",values_to = "val") %>% inner_join(mynames,by = "val") %>% select(c(1,4,2))
mynn_d = mynn_d %>% pivot_longer(cols = matches("^\\d"),names_to = "rank",values_to = "dist")

In [55]:
mynn %>% sample_n(5)
mynn_d %>% sample_n(5)
#mynn_d %>% count(rank)

rowname.x,rowname.y,rank
<chr>,<chr>,<chr>
RUS_Poltavka,Scotland_LIA,170
SRB_BA_Maros,Cossack_Ukrainian,395
RUS_Samara_HG,VK2020_DNK_Langeland_VA,282
FRA_N_BA,BelgianC,316
HRV_MN_Sopot,CHE_IA,251


rowname,rank,dist
<chr>,<chr>,<dbl>
HRV_Pop_CA,773,0.18
Ostrogothic_Crimea_ACD,225,0.11
RUS_Kurma_EBA,199,0.51
Iberia_Northeast_Empuries2,391,0.13
TUR_Tepecik_Ciftlik_N,505,0.18


In [56]:
colnames(mynn) = c("target","population","rank")
colnames(mynn_d) = c("target","rank","distance")
mynn = mynn %>% inner_join(mynn_d) %>% dplyr::filter(rank %in% 1:5)
mynn %>% sample_n(10)

Joining with `by = join_by(target, rank)`


target,population,rank,distance
<chr>,<chr>,<chr>,<dbl>
FRA_Occitanie_EMBA,Iberia_Northeast_BA,3,0.06
CZE_IA_La_Tene,CZE_IA_Hallstatt,2,0.01
CZE_Unetice_EBA,NLD_MBA,5,0.02
IRL_EN_MN,England_N,3,0.02
Wales_MBA,England_MBA,5,0.03
BGR_MLBA,RUS_Krasnoyarsk_MLBA,4,0.03
CZE_N_oWHG,HRV_C_Lasinja,3,0.02
RUS_Baikal_N,RUS_Ust_Belaya_Angara,3,0.05
Italian_Trentino_Alto_Adige,Italian_Veneto,1,0.01


In [57]:
write_csv(mynn,"../Genetics/G25/Data/CSV/top_10_dist_all.csv")

In [58]:
mynn %>% dplyr::filter(str_detect(target,"^Ukrainian.+[^o]$"))

target,population,rank,distance
<chr>,<chr>,<chr>,<dbl>
Ukrainian_Chernihiv,Ukrainian_Rivne,1,0.01
Ukrainian_Chernihiv,Ukrainian_Dnipro,2,0.01
Ukrainian_Chernihiv,Ukrainian_Sumy,3,0.01
Ukrainian_Chernihiv,Ukrainian_Zhytomyr,4,0.01
Ukrainian_Chernihiv,Russian_Orel,5,0.01
Ukrainian_Lviv,Ukrainian_Zakarpattia,1,0.02
Ukrainian_Lviv,Polish,2,0.02
Ukrainian_Lviv,Ukrainian_Sumy,3,0.02
Ukrainian_Lviv,Ukrainian_Zhytomyr,4,0.02


In [97]:
myg = graph_from_data_frame(mynn)

In [98]:
myg

IGRAPH 326f7dd DN-- 995 4975 -- 
+ attr: name (v/c), rank (e/c), distance (e/n)
+ edges from 326f7dd (vertex names):
 [1] ARM_Areni_C     ->Levant_Megiddo_MLBA_o1
 [2] ARM_Areni_C     ->Armenian_Syunik       
 [3] ARM_Areni_C     ->ARM_LBA               
 [4] ARM_Areni_C     ->Ostrogothic_Crimea_ACD
 [5] ARM_Areni_C     ->Greek_Cappadocia      
 [6] ARM_LBA         ->ARM_MBA               
 [7] ARM_LBA         ->ARM_Lchashen_MBA      
 [8] ARM_LBA         ->RUS_Alan_MA           
+ ... omitted several edges

In [99]:
myg = as_tbl_graph(myg)
myg

# A tbl_graph: 995 nodes and 4975 edges
#
# A directed simple graph with 1 component
#
# A tibble: 995 × 1
  name            
  <chr>           
1 ARM_Areni_C     
2 ARM_LBA         
3 ARM_Lchashen_MBA
4 ARM_MBA         
5 AUT_IA_La_Tene  
6 AUT_LBK_N       
# ℹ 989 more rows
#
# A tibble: 4,975 × 4
   from    to rank  distance
  <int> <int> <chr>    <dbl>
1     1   502 1         0.04
2     1   825 2         0.04
3     1     2 3         0.05
# ℹ 4,972 more rows

In [103]:
mypf = function (mygr,wdth = 12,hgth = 12,ppi = 400,bgrd = "gray20") {
    myg = mygr %>% mutate(c_auth = centrality_authority(weights = 1/distance)) 
    print(myg %>% select(name,c_auth) %>% as.data.frame() %>% arrange(desc(c_auth)))
    myp = myg %>% ggraph(layout = 'stress',circular = TRUE) 
    options(repr.plot.width = wdth,repr.plot.height = hgth,repr.plot.res = ppi,repr.plot.bg = bgrd)
    myp + geom_edge_fan(aes(color = distance,alpha = after_stat(index),start_cap = label_rect(node1.name),end_cap = label_rect(node2.name)),strength = 1.25,edge_width = 0.15) + 
        geom_node_label(aes(label = name,color = c_auth),label.size = 0,size = 1,alpha = 0.6,fill = "gray15") + #,repel = TRUE,max.overlaps = 10) + 
        scale_edge_alpha('Edge direction') + 
        scale_edge_color_gradient2("distance",low = "cyan",mid = "snow",high = "orange") + 
        scale_color_gradient2("centrality",low = "cyan",mid = "snow",high = "orange") + 
        theme_void() + 
        theme(plot.background = element_rect("gray20"),
              legend.title = element_text(color = "snow",size = 5),
              legend.text = element_text(color = "snow",size = 4),
              legend.position = "bottom") + #,legend.key.size = unit(4,'pt')) +
        guides(edge_alpha = guide_edge_direction()) + 
        coord_flip()
    ggsave("~/Desktop/plot.pdf",width = wdth,height = hgth,dpi = ppi)
    }

In [118]:
mynames$rowname

[1] "ARM_Areni_C"                        
  [2] "ARM_LBA"                            
  [3] "ARM_Lchashen_MBA"                   
  [4] "ARM_MBA"                            
  [5] "AUT_IA_La_Tene"                     
  [6] "AUT_LBK_N"                          
  [7] "Baltic_EST_BA"                      
  [8] "Baltic_EST_IA"                      
  [9] "Baltic_EST_MA"                      
 [10] "Baltic_EST_Narva"                   
 [11] "Baltic_LTU_BA"                      
 [12] "Baltic_LTU_Meso"                    
 [13] "Baltic_LTU_Narva"                   
 [14] "Baltic_LVA_BA"                      
 [15] "Baltic_LVA_HG"                      
 [16] "Baltic_LVA_MN"                      
 [17] "BEL_GoyetQ116-1"                    
 [18] "BEL_GoyetQ2"                        
 [19] "Bell_Beaker_Bavaria"                
 [20] "Bell_Beaker_CHE"                    
 [21] "Bell_Beaker_CZE"                    
 [22] "Bell_Beaker_CZE_early"              
 [23] "Bell_Beaker_CZE_late"               
 [24] "Bell_Beaker_England"                
 [25] "Bell_Beaker_England_EBA"            
 [26] "Bell_Beaker_England_highEEF"        
 [27] "Bell_Beaker_England_highWHG"        
 [28] "Bell_Beaker_FRA"                    
 [29] "Bell_Beaker_FRA_C"                  
 [30] "Bell_Beaker_HUN"                    
 [31] "Bell_Beaker_HUN_EBA"                
 [32] "Bell_Beaker_Iberia"                 
 [33] "Bell_Beaker_Iberia_C"               
 [34] "Bell_Beaker_ITA"                    
 [35] "Bell_Beaker_Mittelelbe-Saale"       
 [36] "Bell_Beaker_Mittelelbe-Saale_contam"
 [37] "Bell_Beaker_NLD"                    
 [38] "Bell_Beaker_POL"                    
 [39] "Bell_Beaker_Rhine-Main"             
 [40] "Bell_Beaker_Scotland"               
 [41] "BGR_Bacho_Kiro_MUP"                 
 [42] "BGR_Beli_Breyag_EBA"                
 [43] "BGR_C"                              
 [44] "BGR_Dzhulyunitsa_N"                 
 [45] "BGR_EBA"                            
 [46] "BGR_EBA_contam"                     
 [47] "BGR_IA"                             
 [48] "BGR_Krepost_N"                      
 [49] "BGR_Late_C"                         
 [50] "BGR_Middle_C"                       
 [51] "BGR_MLBA"                           
 [52] "BGR_MP_N"                           
 [53] "BGR_N"                              
 [54] "BGR_Varna_C"                        
 [55] "Canary_Islands_Guanche"             
 [56] "Channel_Islands_EIA"                
 [57] "Channel_Islands_IA"                 
 [58] "Channel_Islands_LN"                 
 [59] "Channel_Islands_MN"                 
 [60] "CHE_EBA"                            
 [61] "CHE_FN"                             
 [62] "CHE_FN_steppe_contam"               
 [63] "CHE_IA"                             
 [64] "CHE_LN"                             
 [65] "CHE_LN_contam"                      
 [66] "CHE_LN_steppe"                      
 [67] "CHE_MN"                             
 [68] "Corded_Ware_Baltic"                 
 [69] "Corded_Ware_Baltic_early"           
 [70] "Corded_Ware_CHE"                    
 [71] "Corded_Ware_CZE"                    
 [72] "Corded_Ware_CZE_early"              
 [73] "Corded_Ware_CZE_late"               
 [74] "Corded_Ware_CZE_noSteppe"           
 [75] "Corded_Ware_DEU"                    
 [76] "Corded_Ware_POL"                    
 [77] "Corded_Ware_POL_early"              
 [78] "Corded_Ware_Proto-Unetice_POL"      
 [79] "CZE_Bilina_BA"                      
 [80] "CZE_C"                              
 [81] "CZE_C_oSteppe"                      
 [82] "CZE_Early_Slav"                     
 [83] "CZE_EBA_Unetice"                    
 [84] "CZE_EE"                             
 [85] "CZE_EN_LBK"                         
 [86] "CZE_Hallstatt_Bylany"               
 [87] "CZE_IA_Hallstatt"                   
 [88] "CZE_IA_La_Tene"                     
 [89] "CZE_IA_La_Tene_Hallstatt"           
 [90] "CZE_IA_La_Tene_oFennoscandian"      
 [91] "CZE_Krems_UP"                      

In [136]:
myg %>% convert(to_local_neighborhood,node = which(str_detect(.N()$name,"Slovak")),order = 5,mode = "all") %>% mypf()

                                   name       c_auth
1                       Ukrainian_Rivne 1.000000e+00
2                   Ukrainian_Chernihiv 9.560174e-01
3                    Ukrainian_Zhytomyr 8.424633e-01
4                        Ukrainian_Sumy 7.176534e-01
5                      Ukrainian_Dnipro 7.038894e-01
6                          Russian_Orel 4.728471e-01
7                                Polish 3.865423e-01
8                  DEU_MA_Krakauer_Berg 2.814132e-01
9                            Belarusian 7.990305e-02
10                Ukrainian_Zakarpattia 2.803059e-02
11                       RUS_Sunghir_MA 2.574284e-02
12                             Estonian 2.160849e-02
13                        Russian_Pskov 1.758281e-02
14                VK2020_SWE_Gotland_VA 1.670635e-02
15                                Erzya 1.425295e-02
16                                Czech 8.778148e-03
17                      HUN_Avar_Szolad 7.800091e-03
18                 VK2020_POL_Bodzia_VA 6.6220